# Document ID Migration - Option 4 (Hybrid System)

**Mục đích:** Migrate document IDs từ format cũ sang format mới theo Hybrid System

**Format mới:** `{type_code}-{số_hiệu}/{năm}#{hash_short}`

**Ví dụ:**
- `bidding_untitled` → `FORM-Bidding/2025#bee720`
- `circular_untitled` → `TT-Circular/2025#3be8b6`
- `decree_untitled` → `ND-Decree/2025#95b863`

**Lợi ích:**
- ✅ Human-readable + Machine-friendly
- ✅ Tương thích chuẩn pháp lý VN
- ✅ Đảm bảo uniqueness với hash
- ✅ Dễ query và maintain

---

## Quy trình Migration

1. **Setup & Connect** - Kết nối database
2. **Preview** - Xem trước thay đổi
3. **Backup** - Tạo backup metadata
4. **Execute** - Thực hiện migration
5. **Verify** - Kiểm tra kết quả
6. **Test API** - Test với API endpoints

## Step 1: Setup & Import Libraries

In [1]:
import psycopg
import json
import hashlib
from datetime import datetime
from pathlib import Path
import sys

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.config.models import settings

print("✅ Libraries imported successfully")
print(f"📁 Project root: {project_root}")
print(f"🔗 Database: {settings.database_url.split('@')[1] if '@' in settings.database_url else 'configured'}")

✅ Libraries imported successfully
📁 Project root: /home/sakana/Code/RAG-bidding
🔗 Database: localhost:5432/rag_bidding_v2


## Step 2: Define Migration Functions

In [5]:
def extract_metadata_from_old_id(old_id: str, metadata: dict) -> dict:
    """Extract hoặc infer metadata từ old_id và cmetadata"""
    
    # Lấy document_type từ metadata hoặc old_id
    doc_type = metadata.get("document_type")
    if not doc_type and old_id:
        if "bidding" in old_id.lower():
            doc_type = "bidding"
        elif "circular" in old_id.lower():
            doc_type = "circular"
        elif "decree" in old_id.lower():
            doc_type = "decree"
        elif "law" in old_id.lower():
            doc_type = "law"
        elif "exam" in old_id.lower():
            doc_type = "exam"
        elif "report" in old_id.lower():
            doc_type = "report"
        else:
            doc_type = "document"
    
    # Lấy year từ metadata hoặc timestamp
    year = metadata.get("year")
    if not year and metadata.get("processing_metadata"):
        processed_at = metadata["processing_metadata"].get("last_processed_at", "")
        if processed_at:
            year = processed_at[:4]
    if not year:
        year = "2025"
    
    # Lấy number - dùng type name thay vì "Unknown"
    number = metadata.get("number")
    if not number:
        import re
        match = re.search(r'(\d+)', old_id)
        if match:
            number = match.group(1)
        else:
            number = doc_type.title()
    
    return {
        "type": doc_type or "doc",
        "year": year,
        "number": number
    }


def generate_new_document_id(old_id: str, metadata: dict) -> str:
    """
    Generate new document_id theo Hybrid System
    Format: {type_code}-{number}/{year}#{hash_short}
    """
    
    extracted = extract_metadata_from_old_id(old_id, metadata)
    doc_type = extracted["type"]
    year = extracted["year"]
    number = extracted["number"]
    
    # Type code mapping
    type_code_map = {
        "law": "LAW",
        "decree": "ND",
        "circular": "TT",
        "decision": "QD",
        "bidding": "FORM",
        "report": "RPT",
        "exam": "EXAM",
        "document": "DOC"
    }
    
    type_code = type_code_map.get(doc_type, "DOC")
    
    # Generate hash từ old_id để đảm bảo uniqueness và idempotent
    hash_obj = hashlib.md5(old_id.encode())
    hash_short = hash_obj.hexdigest()[:6]
    
    new_id = f"{type_code}-{number}/{year}#{hash_short}"
    
    return new_id


print("✅ Migration functions defined")

✅ Migration functions defined


## Step 3: Connect to Database & Preview Current Data

In [6]:
# Connect to database
# Use database_url from settings (convert SQLAlchemy format to psycopg format)
conn_str = settings.database_url.replace("postgresql+psycopg://", "postgresql://")

print("🔌 Connecting to database...")
print(f"🔗 Connection: {conn_str.split('@')[1] if '@' in conn_str else 'configured'}")
conn = psycopg.connect(conn_str)
cursor = conn.cursor()

# Query all unique document_ids with their chunk counts
query = """
SELECT 
    cmetadata->>'document_id' as document_id,
    COUNT(*) as chunk_count,
    MIN(cmetadata->>'doc_type') as doc_type,
    MIN(cmetadata->>'processing_metadata') as processing_metadata_sample
FROM langchain_pg_embedding
WHERE cmetadata->>'document_id' IS NOT NULL
GROUP BY cmetadata->>'document_id'
ORDER BY document_id;
"""

cursor.execute(query)
results = cursor.fetchall()

print(f"\n📊 Found {len(results)} documents with {sum(r[1] for r in results)} total chunks\n")
print("=" * 100)
print(f"{'OLD DOCUMENT_ID':<30} {'NEW DOCUMENT_ID':<30} {'TYPE':<8} {'CHUNKS':<8}")
print("=" * 100)

migration_preview = []
for old_id, chunk_count, doc_type, processing_metadata in results:
    # Parse processing_metadata if it exists
    metadata = {}
    if processing_metadata:
        try:
            import json
            metadata = json.loads(processing_metadata)
        except:
            metadata = {}
    
    # Generate new document_id
    new_id = generate_new_document_id(old_id, metadata)
    
    # Determine doc type for display
    inferred_meta = extract_metadata_from_old_id(old_id, metadata)
    display_type = inferred_meta.get('doc_type', 'unknown')
    
    print(f"{old_id:<30} {new_id:<30} {display_type:<8} {chunk_count:<8}")
    
    migration_preview.append({
        'old_id': old_id,
        'new_id': new_id,
        'doc_type': display_type,
        'chunk_count': chunk_count
    })

print("=" * 100)
print(f"\n✅ Preview complete: {len(migration_preview)} documents ready for migration")

# Close connection (we'll reopen for migration)
cursor.close()
conn.close()
print("🔌 Database connection closed")

🔌 Connecting to database...
🔗 Connection: localhost:5432/rag_bidding_v2

📊 Found 5 documents with 4708 total chunks

OLD DOCUMENT_ID                NEW DOCUMENT_ID                TYPE     CHUNKS  
bidding_untitled               FORM-Bidding/2025#bee720       unknown  2831    
circular_untitled              TT-Circular/2025#3be8b6        unknown  123     
decision_untitled              DOC-Document/2025#787999       unknown  5       
decree_untitled                ND-Decree/2025#95b863          unknown  595     
law_untitled                   LAW-Law/2025#cd5116            unknown  1154    

✅ Preview complete: 5 documents ready for migration
🔌 Database connection closed


## Step 4: Backup Current Data

**⚠️ IMPORTANT:** Trước khi migrate, chúng ta sẽ backup toàn bộ cmetadata hiện tại để có thể rollback nếu cần.

In [8]:
# Create backup directory (sử dụng Path.cwd() thay vì __file__ trong notebook)
backup_dir = Path.cwd() / "backups"
backup_dir.mkdir(exist_ok=True)

# Generate backup filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_file = backup_dir / f"document_id_backup_{timestamp}.json"

print(f"💾 Creating backup at: {backup_file}")

# Connect and fetch all metadata
conn = psycopg.connect(conn_str)
cursor = conn.cursor()

backup_query = """
SELECT 
    id,
    cmetadata->>'document_id' as document_id,
    cmetadata
FROM langchain_pg_embedding
WHERE cmetadata->>'document_id' IS NOT NULL
ORDER BY id;
"""

cursor.execute(backup_query)
backup_data = cursor.fetchall()

# Save to JSON
backup_records = []
for row_id, doc_id, cmetadata in backup_data:
    backup_records.append({
        'id': row_id,
        'document_id': doc_id,
        'cmetadata': cmetadata
    })

with open(backup_file, 'w', encoding='utf-8') as f:
    json.dump({
        'timestamp': timestamp,
        'total_chunks': len(backup_records),
        'records': backup_records
    }, f, indent=2, ensure_ascii=False)

cursor.close()
conn.close()

print(f"✅ Backup created: {len(backup_records)} chunks saved")
print(f"📁 Backup location: {backup_file}")
print("\n⚠️  To rollback, run this SQL:")
print(f"-- Restore from backup: {backup_file.name}")
print("-- UPDATE langchain_pg_embedding SET cmetadata = backup_cmetadata WHERE id = backup_id")

💾 Creating backup at: /home/sakana/Code/RAG-bidding/notebooks/backups/document_id_backup_20251109_154532.json
✅ Backup created: 4708 chunks saved
📁 Backup location: /home/sakana/Code/RAG-bidding/notebooks/backups/document_id_backup_20251109_154532.json

⚠️  To rollback, run this SQL:
-- Restore from backup: document_id_backup_20251109_154532.json
-- UPDATE langchain_pg_embedding SET cmetadata = backup_cmetadata WHERE id = backup_id
✅ Backup created: 4708 chunks saved
📁 Backup location: /home/sakana/Code/RAG-bidding/notebooks/backups/document_id_backup_20251109_154532.json

⚠️  To rollback, run this SQL:
-- Restore from backup: document_id_backup_20251109_154532.json
-- UPDATE langchain_pg_embedding SET cmetadata = backup_cmetadata WHERE id = backup_id


## Step 5: Execute Migration

**⚠️ CRITICAL:** Đây là bước thay đổi database thật. Hãy chắc chắn bạn đã:
- ✅ Review preview ở Step 3
- ✅ Đã có backup ở Step 4
- ✅ Sẵn sàng thực hiện migration

**Để thực hiện migration, set `CONFIRM_MIGRATION = True` trong cell dưới.**

In [10]:
# ⚠️  SET THIS TO True TO CONFIRM MIGRATION
CONFIRM_MIGRATION = True

if not CONFIRM_MIGRATION:
    print("❌ Migration NOT confirmed. Set CONFIRM_MIGRATION = True to proceed.")
else:
    print("🚀 Starting migration...\n")
    
    # Connect to database
    conn = psycopg.connect(conn_str)
    cursor = conn.cursor()
    
    try:
        total_updated = 0
        
        for item in migration_preview:
            old_id = item['old_id']
            new_id = item['new_id']
            chunk_count = item['chunk_count']
            
            print(f"📝 Migrating: {old_id} → {new_id} ({chunk_count} chunks)...")
            
            # Update document_id in cmetadata
            update_query = """
            UPDATE langchain_pg_embedding
            SET cmetadata = jsonb_set(
                cmetadata, 
                '{document_id}', 
                to_jsonb(%s::text)
            )
            WHERE cmetadata->>'document_id' = %s;
            """
            
            cursor.execute(update_query, (new_id, old_id))
            updated_count = cursor.rowcount
            total_updated += updated_count
            
            print(f"   ✅ Updated {updated_count} chunks")
        
        # Commit all changes
        conn.commit()
        print(f"\n🎉 Migration complete! Total chunks updated: {total_updated}")
        print(f"✅ All changes committed to database")
        
    except Exception as e:
        conn.rollback()
        print(f"\n❌ Error during migration: {e}")
        print("🔄 All changes rolled back")
        raise
    
    finally:
        cursor.close()
        conn.close()
        print("🔌 Database connection closed")

🚀 Starting migration...

📝 Migrating: bidding_untitled → FORM-Bidding/2025#bee720 (2831 chunks)...
   ✅ Updated 2831 chunks
📝 Migrating: circular_untitled → TT-Circular/2025#3be8b6 (123 chunks)...
   ✅ Updated 123 chunks
📝 Migrating: decision_untitled → DOC-Document/2025#787999 (5 chunks)...
   ✅ Updated 5 chunks
📝 Migrating: decree_untitled → ND-Decree/2025#95b863 (595 chunks)...
   ✅ Updated 595 chunks
📝 Migrating: law_untitled → LAW-Law/2025#cd5116 (1154 chunks)...
   ✅ Updated 1154 chunks

🎉 Migration complete! Total chunks updated: 4708
✅ All changes committed to database
🔌 Database connection closed
   ✅ Updated 2831 chunks
📝 Migrating: circular_untitled → TT-Circular/2025#3be8b6 (123 chunks)...
   ✅ Updated 123 chunks
📝 Migrating: decision_untitled → DOC-Document/2025#787999 (5 chunks)...
   ✅ Updated 5 chunks
📝 Migrating: decree_untitled → ND-Decree/2025#95b863 (595 chunks)...
   ✅ Updated 595 chunks
📝 Migrating: law_untitled → LAW-Law/2025#cd5116 (1154 chunks)...
   ✅ Updated 

## Step 6: Verify Migration Results

Kiểm tra xem migration đã thành công chưa bằng cách query database.

In [11]:
# Connect and verify
conn = psycopg.connect(conn_str)
cursor = conn.cursor()

print("🔍 Verifying migration results...\n")

# Check all document_ids now in database
verify_query = """
SELECT 
    cmetadata->>'document_id' as document_id,
    COUNT(*) as chunk_count,
    MIN(cmetadata->>'doc_type') as doc_type
FROM langchain_pg_embedding
WHERE cmetadata->>'document_id' IS NOT NULL
GROUP BY cmetadata->>'document_id'
ORDER BY document_id;
"""

cursor.execute(verify_query)
results = cursor.fetchall()

print(f"📊 Current database state: {len(results)} documents\n")
print("=" * 80)
print(f"{'DOCUMENT_ID':<35} {'TYPE':<10} {'CHUNKS':<10}")
print("=" * 80)

total_chunks = 0
for doc_id, chunk_count, doc_type in results:
    print(f"{doc_id:<35} {doc_type or 'unknown':<10} {chunk_count:<10}")
    total_chunks += chunk_count

print("=" * 80)
print(f"Total: {total_chunks} chunks across {len(results)} documents\n")

# Check for any old-format IDs remaining
check_old_format = """
SELECT COUNT(*) 
FROM langchain_pg_embedding
WHERE cmetadata->>'document_id' LIKE '%_untitled'
   OR cmetadata->>'document_id' NOT LIKE '%#%';
"""

cursor.execute(check_old_format)
old_format_count = cursor.fetchone()[0]

if old_format_count > 0:
    print(f"⚠️  Warning: {old_format_count} chunks still have old-format document_ids")
else:
    print("✅ All document_ids successfully migrated to new format!")

# Sample some chunks to verify metadata integrity
sample_query = """
SELECT 
    id,
    cmetadata->>'document_id' as document_id,
    cmetadata->>'doc_type' as doc_type,
    cmetadata->>'chunk_index' as chunk_index
FROM langchain_pg_embedding
LIMIT 5;
"""

cursor.execute(sample_query)
samples = cursor.fetchall()

print("\n📋 Sample chunks (first 5):")
print("=" * 80)
for row_id, doc_id, doc_type, chunk_idx in samples:
    print(f"ID: {row_id} | Doc: {doc_id} | Type: {doc_type} | Chunk: {chunk_idx}")
print("=" * 80)

cursor.close()
conn.close()

print("\n✅ Verification complete!")

🔍 Verifying migration results...

📊 Current database state: 5 documents

DOCUMENT_ID                         TYPE       CHUNKS    
DOC-Document/2025#787999            unknown    5         
FORM-Bidding/2025#bee720            unknown    2831      
LAW-Law/2025#cd5116                 unknown    1154      
ND-Decree/2025#95b863               unknown    595       
TT-Circular/2025#3be8b6             unknown    123       
Total: 4708 chunks across 5 documents

✅ All document_ids successfully migrated to new format!

📋 Sample chunks (first 5):
ID: 4af1c105-4ce2-43a6-b304-244a6d16ee15 | Doc: FORM-Bidding/2025#bee720 | Type: None | Chunk: 102
ID: 82d0d255-4401-470d-9805-02ffe2a677a9 | Doc: FORM-Bidding/2025#bee720 | Type: None | Chunk: 66
ID: 748f9e32-7127-43c4-b575-71cea7d40e78 | Doc: FORM-Bidding/2025#bee720 | Type: None | Chunk: 46
ID: 8e742f0e-8a04-4cfe-b7a7-2548a93179fa | Doc: FORM-Bidding/2025#bee720 | Type: None | Chunk: 1
ID: ca5d8bad-ed7f-475b-ab69-44a317a611b1 | Doc: FORM-Bidding/202

## Step 7: Test API with New Document IDs

Kiểm tra API endpoints hoạt động đúng với document_id format mới.

**Lưu ý:** Document IDs có ký tự `#` cần được URL-encode khi gọi API:
- `ND-43/2022#a7f3c9` → `ND-43/2022%23a7f3c9`

In [13]:
import urllib.parse
import requests

# Assume API is running at localhost:8000
API_BASE_URL = "http://localhost:8000/api"

print("🧪 Testing API with new document IDs...\n")

# Get a sample new document_id from migration_preview
if migration_preview:
    sample_doc = migration_preview[0]
    test_doc_id = sample_doc['new_id']
    
    print(f"📝 Testing with document: {test_doc_id}")
    print(f"   (Originally: {sample_doc['old_id']})")
    print(f"   Type: {sample_doc['doc_type']}, Chunks: {sample_doc['chunk_count']}\n")
    
    # URL encode the document_id (important for # and / symbols)
    encoded_doc_id = urllib.parse.quote(test_doc_id, safe='')
    
    print(f"🔗 URL-encoded: {encoded_doc_id}\n")
    
    # Test 1: GET document status (using query parameter)
    print("=" * 80)
    print("TEST 1: GET /api/document-status?document_id={document_id}")
    print("=" * 80)
    
    try:
        response = requests.get(
            f"{API_BASE_URL}/document-status",
            params={"document_id": test_doc_id}  # Use query param, requests will encode
        )
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ GET request successful!")
            print(f"Response: {json.dumps(data, indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ GET request failed: {response.text}")
    except Exception as e:
        print(f"❌ Error calling API: {e}")
        print("⚠️  Make sure the API server is running: uvicorn src.api.main:app --reload")
    
    print("\n")
    
    # Test 2: POST update document status
    print("=" * 80)
    print("TEST 2: POST /api/document-status/update")
    print("=" * 80)
    
    try:
        update_data = {
            "document_id": test_doc_id,  # Don't encode in JSON body
            "new_status": "active",
            "reason": "Testing new document_id format after migration",
            "notes": "Migration successful - testing API compatibility"
        }
        
        response = requests.post(
            f"{API_BASE_URL}/document-status/update",
            json=update_data
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ POST request successful!")
            print(f"Response: {json.dumps(data, indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ POST request failed: {response.text}")
    except Exception as e:
        print(f"❌ Error calling API: {e}")
        print("⚠️  Make sure the API server is running: uvicorn src.api.main:app --reload")
    
    print("=" * 80)
    print("\n✅ API testing complete!")
    
else:
    print("❌ No migration preview data available. Run Step 3 first.")

🧪 Testing API with new document IDs...

📝 Testing with document: FORM-Bidding/2025#bee720
   (Originally: bidding_untitled)
   Type: unknown, Chunks: 2831

🔗 URL-encoded: FORM-Bidding%2F2025%23bee720

TEST 1: GET /api/document-status?document_id={document_id}
Status Code: 200
✅ GET request successful!
Response: {
  "document_id": "FORM-Bidding/2025#bee720",
  "current_status": "active",
  "chunk_count": 2831,
  "last_updated": "2025-11-09T15:46:40.002664",
  "superseded_by": "bidding_new_2024"
}


TEST 2: POST /api/document-status/update
Status Code: 200
✅ POST request successful!
Response: {
  "success": true,
  "message": "Đã cập nhật status cho 2831 chunks",
  "document_id": "FORM-Bidding/2025#bee720",
  "chunks_updated": 2831,
  "old_status": "active",
  "new_status": "active"
}

✅ API testing complete!
Status Code: 200
✅ POST request successful!
Response: {
  "success": true,
  "message": "Đã cập nhật status cho 2831 chunks",
  "document_id": "FORM-Bidding/2025#bee720",
  "chunks_

## 🎉 Migration Complete!

### Tóm tắt quy trình:

1. ✅ **Setup & Import** - Đã load các thư viện và kết nối database
2. ✅ **Define Functions** - Đã tạo các hàm migration logic
3. ✅ **Preview** - Đã xem trước các thay đổi (5 documents → 4708 chunks)
4. ✅ **Backup** - Đã backup toàn bộ metadata hiện tại
5. ✅ **Execute** - Đã thực hiện migration (nếu CONFIRM_MIGRATION = True)
6. ✅ **Verify** - Đã kiểm tra kết quả migration
7. ✅ **Test API** - Đã test API với document_id mới

### Document ID Format mới:

```
{TYPE_CODE}-{Number}/{Year}#{Hash}
```

**Ví dụ:**
- `ND-43/2022#a7f3c9` - Nghị định 43/2022
- `TT-15/2023#65aabb` - Thông tư 15/2023
- `LAW-Law/2025#cd5116` - Luật (không có số cụ thể)

### Lưu ý tiếp theo:

1. **Cập nhật preprocessing pipeline** để sử dụng `DocumentIDGenerator` cho các văn bản mới
2. **Update documentation** về format document_id mới
3. **Thông báo team** về sự thay đổi này
4. **Monitor API** để đảm bảo không có issues với URL encoding (#)

### Rollback (nếu cần):

Nếu cần quay lại document_id cũ, sử dụng file backup trong `notebooks/backups/` và chạy:

```python
# Load backup file
with open('backups/document_id_backup_TIMESTAMP.json', 'r') as f:
    backup = json.load(f)

# Restore each record
for record in backup['records']:
    cursor.execute("""
        UPDATE langchain_pg_embedding
        SET cmetadata = %s
        WHERE id = %s
    """, (json.dumps(record['cmetadata']), record['id']))
```